<a href="https://colab.research.google.com/github/knuteriksen/VisionPorComputador/blob/main/GenderRecognition100_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check which GPU Google gives you - just for fun

In [ ]:
!nvidia-smi -L

Install keras

In [ ]:
!pip3 install keras

Imports needed


In [ ]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD, Adam

from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator
import os


Definitions, and extractions and preprprocessing of training and test dada

In [ ]:
## Definitions
num_classes = 2

# Download training and test sets
os.system('wget https://www.dropbox.com/s/zcwlujrtz3izcw8/gender.tgz')
os.system('tar xvzf gender.tgz')


# Load
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


# Print out information
print("X train: ", x_train.shape)
print("Y train: ", y_train.shape)
print("X test:  ", x_test.shape)
print("Y test:  ", y_test.shape)

## Transforms
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_train /= 255
x_test /= 255

print("X train: ", x_train.shape)
print("Y train: ", y_train.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Building of model

In [ ]:
model = Sequential()
model.add(keras.Input(shape=x_train.shape[1:]))

model.add(Conv2D(filters=16, kernel_size=3, strides=1, activation="relu", padding="same"))
model.add(BN())
model.add(Conv2D(filters=16, kernel_size=3, strides=1, activation="relu", padding="same"))
model.add(BN())
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=32, kernel_size=3, strides=1, activation="relu", padding="same"))
model.add(BN())
model.add(Conv2D(filters=32, kernel_size=3, strides=1, activation="relu", padding="same"))
model.add(BN())
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation="relu", padding="same"))
model.add(BN())
model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation="relu", padding="same"))
model.add(BN())
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

Display model

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

Compile model

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(learning_rate=0.0001),
              metrics=["accuracy"])


datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  zoom_range=[1.0,1.2],
  horizontal_flip=True
  )

def multiple_data_generator(generator, X,Y,bs):
    genX = generator.flow(X, Y, batch_size=bs)
    while True:
      [Xi,Yi] = genX.next()
      yield   Xi,Yi

Training

In [ ]:
batch_size = 73

history=model.fit(x=multiple_data_generator(datagen, x_train, y_train, batch_size),
                  epochs=150,
                  steps_per_epoch = len(x_train) / batch_size,                 
                  validation_data=(x_test, y_test),
                  verbose=1
)

Testing

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)

f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
ax1.set_yscale('log')
ax1.plot(history.history['loss'], label="loss")
ax1.plot(history.history['val_loss'], label="val_loss")
ax1.legend()

ax2.plot(history.history['accuracy'], label="accuracy")
ax2.plot(history.history['val_accuracy'], label="validation accuracy")
ax2.legend()

plt.setp(ax1, xlabel='Epochs')
plt.setp(ax2, xlabel='Epochs')
f.suptitle('Gender recognition', fontsize='x-large')
plt.show();